In [1]:
import random

from evaluator import evaluate_with_ollama, print_results
from tools.get_current_weather import GetCurrentWeatherTool
from tools.get_car_temp_setpoint import GetCarTemperatureSetpointTool
from tools.read_news import ReadNewsTool
from tools.set_car_temp_setpoint import SetCarTemperatureSetpointTool
from tools.tune_radio_tool import TuneRadioTool

In [2]:
model_tags = ["phi3:3.8b-mini-instruct-4k-q4_K_M", "llama3:8b-instruct-q4_0", "wizardlm2:7b-q4_0"]
tools = [GetCurrentWeatherTool(), SetCarTemperatureSetpointTool(), GetCarTemperatureSetpointTool(), TuneRadioTool(), ReadNewsTool()]

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/timothypark/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/timothypark/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
reproducability_seed = random.randint(1, 100000)
print(f"reproducability_seed: {reproducability_seed}")

reproducability_seed: 8920


In [4]:
scenarios = []
evaluations_per_tool = 100

for tool in tools:
    for i in range(evaluations_per_tool):
        scenarios.append(tool.generate_scenario())

random.shuffle(scenarios)

In [12]:
# correctness by model
model_results = {}
for model_tag in model_tags:
    model_results[model_tag] = evaluate_with_ollama(model_tag, tools, scenarios)

arguments failed: {'subject': 'latest news on college football'} vs. expected {'subject': 'college football'} for prompt What's the latest news on college football?
arguments failed: {'subject': 'stock market news'} vs. expected {'subject': 'stock market'} for prompt What's new with the stock market?
arguments failed: {'subject': 'college football news'} vs. expected {'subject': 'college football'} for prompt Could you tell me the latest about college football?
arguments failed: {'subject': 'college football news'} vs. expected {'subject': 'college football'} for prompt What are the headlines about college football?
arguments failed: {'type': 'object', 'properties': {}, 'required': []} vs. expected {} for prompt What is the current car temperature?
arguments failed: {'subject': 'college football news'} vs. expected {'subject': 'college football'} for prompt What's new with college football?
arguments failed: {'subject': 'Premier League news'} vs. expected {'subject': 'Premier League'} 

In [15]:
for model_tag in model_results:
    print(model_tag)
    print(print_results(tools, model_results[model_tag]))

with open('results/m1-max.json', 'w') as file:
    json.dump(model_results, file)

phi3:3.8b-mini-instruct-4k-q4_K_M
tool                            passed    total    %    min (ms)    mean (ms)    median (ms)    max (ms)
----------------------------  --------  -------  ---  ----------  -----------  -------------  ----------
get_current_weather                100      100  100     599.367      1195.54        1289.17     1624.79
set_car_temperature_setpoint       100      100  100     941.641      1541.11        1621.31     1706.46
get_car_temperature_setpoint        99      100   99     561.873      1238.23        1352        1866.58
set_radio_station                  100      100  100     641.853      1523.92        1584.82     1730.12
read_news                           92      100   92     574.628      1306.16        1499.21     1560.94
None
llama3:8b-instruct-q4_0
tool                            passed    total    %    min (ms)    mean (ms)    median (ms)    max (ms)
----------------------------  --------  -------  ---  ----------  -----------  -------------  ---

In [ ]:
# latency by model

In [ ]:
# latency distribution for a tool using phi3

In [ ]:
# Are there user inputs that are more likely to fail?
# How do scores vary with different number of tools?
# How does latency vary with different tools?
# How does hardware effect latency (4800 vs. M1 Max vs M4)?
# How does latency vary by model? (phi3 vs. phi3 4bit vs. llama3?)
# How does correctness vary by model?
# What is the memory usage by model?